<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/examples/llm/gradient_base_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gradient Base Model

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [ ]:
!pip install llama-index

In [ ]:
%pip install llama-index --quiet
%pip install gradientai --quiet

In [ ]:
import os

os.environ["GRADIENT_ACCESS_TOKEN"] = "{GRADIENT_ACCESS_TOKEN}"
os.environ["GRADIENT_WORKSPACE_ID"] = "{GRADIENT_WORKSPACE_ID}"

## Flow 1: Query Gradient LLM directly

In [ ]:
from llama_index.llms import GradientBaseModelLLM

llm = GradientBaseModelLLM(
    base_model_slug="llama2-7b-chat",
    max_tokens=400,
)

In [ ]:
result = llm.complete("Can you tell me about large language models?")
print(result)

## Flow 2: Retrieval Augmented Generation (RAG) with Gradient LLM

In [ ]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.embeddings import LangchainEmbedding
from langchain.embeddings import HuggingFaceEmbeddings

#### Download Data

In [ ]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

### Load Documents

In [ ]:
documents = SimpleDirectoryReader("./data/paul_graham/").load_data()

### Configure Gradient LLM

In [ ]:
embed_model = LangchainEmbedding(HuggingFaceEmbeddings())
service_context = ServiceContext.from_defaults(
    chunk_size=1024, llm=llm, embed_model=embed_model
)

### Setup and Query Index

In [ ]:
index = VectorStoreIndex.from_documents(
    documents, service_context=service_context
)
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query(
    "What did the author do after his time at Y Combinator?"
)
print(response)